Keyphrase Extraction using Word2Vec with Hierarchical Clustering

1. Importing Libraries

In [1]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import nltk,re, pprint
from nltk.chunk.regexp import *
from nltk.corpus import stopwords
from nltk.chunk.regexp import tag_pattern2re_pattern 

ModuleNotFoundError: No module named 'gensim'

2. Text Processing

In [ ]:
# Parameters Set
n_clusters = 100    # No. Of Clusters
stop_words = set(stopwords.words('english'))
Actual_Summary = "output.txt"


# Text Processing : Handling StopWords, Tokenization
def text_processing(file):
    f = open(file, 'r', encoding='utf-8')
    text = f.read()
    
    #Tokenization : Convert to sentences and then to words
    sentences = nltk.sent_tokenize(text)
    data = []
    
    for sent in sentences:
        words = nltk.word_tokenize(sent)
        words = [w.lower() for w in words if len(w)>2 and w not in stop_words]
        words = [w.lower() for w in words if w != "the"]
        data.append(words)
        
    return data


text = text_processing("input.txt")
print("Text Processing : done")


print(text)

count_frequency = {}
for line in text:
    for w in line:
        count_frequency[w] = 0
        
for line in text:
    for w in line:
        count_frequency[w] = count_frequency[w] + 1
            
candidate_term = len(count_frequency.keys())

3. Implement Word2Vec Model

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(text, size=300, window = 10, min_count=1)
words = list(model.wv.vocab)

print("Word2Vec Model : done")

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,300), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()
    
display_closestwords_tsnescatterplot(model, words[0])


4. Hierarchical Clustering Applied

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

l = linkage(model.wv.syn0, method='complete', metric='seuclidean')

# calculate full dendrogram
plt.figure(figsize=(250, 100))
plt.title('Hierarchical Clustering Dendrogram')
plt.ylabel('word')
plt.xlabel('distance')

dendrogram(
    l,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=16.,  # font size for the x axis labels
    orientation='left',
    leaf_label_func=lambda v: str(model.wv.index2word[v])
)
# plt.show()
print("Hierarchical Clustering : done")


5. Applied K-Means and Regular Expression Parser

In [ ]:
from scipy.spatial import distance
from sklearn.cluster import KMeans
import numpy as np

n_clusters = int(candidate_term*4/5)

print("\n clusters : ", n_clusters)

from sklearn.cluster import AgglomerativeClustering
hc = AgglomerativeClustering(n_clusters, affinity="euclidean", linkage='complete')
y_hc = hc.fit_predict(model.wv.syn0)

print("***************************************************************************************************")

# Extracting The Keyword by choosing the mean of cluster
summary=[]
def Extracted_Keywords(clus, clus_name):
    
    kmeans = KMeans(n_clusters=1, random_state=0).fit(clus)
    labels = kmeans.labels_
    mean = kmeans.cluster_centers_.reshape(300,1)
    min_to_centre = []
    
    for i in clus:
        Y = distance.cdist(i.reshape(300,1), mean, 'euclidean')
        sum_Y = np.sum(Y)
        min_to_centre.append(sum_Y)

    min_value = min(min_to_centre)
    ind_value = np.argmin(min_to_centre)
    
#     print(ind_value)
#     print(clus_name[ind_value])
    summary.append(clus_name[ind_value])

for j in range(0,n_clusters):
    clus = []
    clus_name = []
    count = 0
    for i in range(len(y_hc)):
        if(y_hc[i] == j):
            count += 1
            clus.append(model[words[i]])
            clus_name.append(words[i])
    
    Extracted_Keywords(clus, clus_name)


    
print("done")


In [ ]:
import collections
summary_frequency = {}

for word in summary:
    summary_frequency[word] = count_frequency[word]
    
od = sorted(summary_frequency.items(), key=lambda x: x[1])
od = od[::-1]

result = []

for i in range(0,20):
    result.append(od[i][0])

print(od)

    
    
# Regular Expression Parser
grammar = "chunk:{(<JJ>)*(<NN | NNS | NNP>)+} " 
chunkParser= RegexpParser(grammar)
tagged = nltk.pos_tag(result)
result = chunkParser.parse(tagged)
for subtree in result.subtrees():
    print(subtree)
# result.draw()


In [ ]:
right_lis=[]
def getNodes(parent):
    print(parent.label())
    for node in parent:
      
        if type(node) is nltk.Tree:
            if node.label() == 'chunk':
                print("hii")
                #print(node.leaves())
                right_lis.append(node.leaves())
                #print("Sentence:", " ".join(node.leaves()))  
                
            
            else:
                print(node.label())
        else:
            pass
        print(node)  
        
getNodes(result)
print("done")

In [ ]:

keyphrase=[]
str1=''
for i in range(len(right_lis)):
    for j in range(len(right_lis[i])):
        for k in range(len(right_lis[i][j])-1):
            if right_lis[i][j][k+1] != 'JJ':
             
                str1=str1+right_lis[i][j][k]
                keyphrase.append(str1)
                str1=''
            else:
                str1=str1+right_lis[i][j][k] + ' '
    str1=' '
print(right_lis)
print("\n\n")

print("Keyphrases : ",keyphrase)

6. Results : Precision, Recall, F-Measure

In [ ]:
from rouge import Rouge 

def Calculate_Score():
    str2 = ""
    for i in summary:
        str2 = str2 + i
        str2 = str2 + " "
    
    print("Predicted Summary : \n")
    print(str2)

    print("\n\n")
    
    
    f = open(Actual_Summary, 'r', encoding='utf-8')
    text = f.read()
    
    print("Actual Summary : \n")
    print(text)

    rouge = Rouge()
    scores = rouge.get_scores(text, str2)
    print("\n\n")
    
    print("F-Measure : ", scores[0]['rouge-1']['f'])
    print("Precision : ", scores[0]['rouge-1']['p'])
    print("Recall    : ", scores[0]['rouge-1']['r'])
    
Calculate_Score()